In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from surv_ai.lib.log import logger, AgentLogLevel
import logging

logger.set_log_level(AgentLogLevel.INTERNAL)
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from surv_ai import (
    GPTClient,
    Survey,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge
)

client = GPTClient(os.environ["OPEN_AI_API_KEY"])

survey = Survey(
    client=client,
    tool_belt=ToolBelt(
        client,
        tools=[
            GoogleCustomSearchTool(
                client,
                os.environ["GOOGLE_API_KEY"],
                os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                start_date="2023-01-01",
                end_date="2023-05-01",
                n_pages=10,
            )
        ]
    ),
    max_knowledge_per_agent=3,
    n_agents=10,
    base_knowledge=[
        Knowledge(
            text="It is currently 2023/05/01, all the articles are from 2023.",
            source="Additional context",
        ),
    ],
)

await survey.conduct(
    "California experienced a significant amount of rainfall this winter.",
)  # This should always returns high a confidence agreement.

In [ ]:
from surv_ai import (
    GPTClient,
    Survey,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge
)

client = GPTClient(os.environ["OPEN_AI_API_KEY"])

survey = Survey(
    client=client,
    tool_belt=ToolBelt(
        client,
        tools=[
            GoogleCustomSearchTool(
                client,
                os.environ["GOOGLE_API_KEY"],
                os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                n_pages=10,
                start_date="2023-01-01",
                end_date="2023-05-01",
            )
        ]
    ),
    n_agents=10,
    base_knowledge=[
        Knowledge(
            text="It is currently 2023/05/01, all the articles are from 2023.",
            source="Additional context",
        ),
    ],
)

await survey.conduct(
    "California experienced little rainfall this winter.",
)  # This should always returns a high confidence disagreement.

In [ ]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)


client = GPTClient(os.environ["OPEN_AI_API_KEY"])

date_ranges = [
    ('2022-05-01', '2022-06-01'),
    ('2022-06-01', '2022-07-01'),
    ('2022-07-01', '2022-08-01'),
    ('2022-08-01', '2022-09-01'),
    ('2022-09-01', '2022-10-01'),
    ('2022-10-01', '2022-11-05'),
]

model = Model(
    Survey,
    parameters=[
        SurveyParameter(
            independent_variable=range[1],
            kwargs={
                "client": client,
                "n_agents": 100,
                "max_knowledge_per_agent":20,
                "max_concurrency": 10,
                "tool_belt": ToolBelt(
                    client,
                    [
                        GoogleCustomSearchTool(
                            client,
                            os.environ["GOOGLE_API_KEY"],
                            os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                            n_pages=20,
                            start_date=range[0],
                            end_date=range[1]
                        ),
                    ],
                ),
                "base_knowledge": [
                    Knowledge(
                        text=f"It is currently {range[0]}. The included articles were published between {range[0]} and {range[1]}",
                        source="Additional context",
                    ),
                ],
            },
        )
        for range in date_ranges
    ],
)

results = await model.build(
    "Democrats are favored to maintain control of the Senate in the 2022 November Midterm elections.",
)

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot

COLOR_MODEL = "#69b3a2"
COLOR_538 = "#3399e6"
  
variables = model.get_plot_variables(results)


# plotting the points 
pyplot.plot([datetime(int(r[0:4]), int(r[5:7]), int(r[8:10])) for r in variables[0]], variables[1], color=COLOR_MODEL, label="surv_ai Model")
pyplot.plot([datetime(int(r[0:4]), int(r[4:6]), int(r[6:8])) for r in ['20220601', '20220701', '20220801', '20220901', '20221001', '20221101']], [1 - r for r in [0.60, 0.54, 0.44, 0.32, 0.32, 0.55]], color=COLOR_538, label="FiveThirtyEight.com Model")

pyplot.ylabel("Agreement (n_agents=100)")
pyplot.xlabel("Date")

pyplot.legend(loc="upper left")
  
# giving a title to my graph
pyplot.title('"Democrats are favored to maintain control of the Senate in the 2022 November Midterm elections."')
  
# function to show the plot
pyplot.show()

In [ ]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)
client = GPTClient(os.environ["OPEN_AI_API_KEY"])

date_ranges = [
    ('2021-09-01', '2022-01-01'),
    ('2022-01-01', '2022-03-01'),
    ('2022-03-01', '2022-06-01'),
    ('2022-06-01', '2022-09-01'),
    ('2022-09-01', '2023-01-01'),
    ('2023-01-01', '2023-03-01'),
    ('2023-03-01', '2023-06-01'),
]

model = Model(
    Survey,
    parameters=[
        SurveyParameter(
            independent_variable=range[1],
            kwargs={
                "client": client,
                "n_agents": 50,
                "max_knowledge_per_agent": 20,
                "max_concurrency": 20,
                "tool_belt": ToolBelt(
                    client,
                    [
                        GoogleCustomSearchTool(
                            client,
                            os.environ["GOOGLE_API_KEY"],
                            os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                            n_pages=20,
                            start_date=range[0],
                            end_date=range[1]
                        ),
                    ],
                ),
                "base_knowledge": [
                    Knowledge(
                        text=f"It is currently {range[0]}. The included articles were published between {range[0]} and {range[1]}",
                        source="Additional context",
                    ),
                ],
            },
        )
        for range in date_ranges
    ],
)

results = await model.build(
    "The United States economy looks like it is heading for a recession.",
)

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import pyplot

# results_1 = results

COLOR_MODEL = "#69b3a2"
COLOR_CCI = "#3399e6"
  
variables = model.get_plot_variables(results_1)
fig, ax1 = plt.subplots()
fig.set_size_inches(10.5, 5)

ax2 = ax1.twinx()

# plotting the points 
ax1.plot([datetime(int(r[0:4]), int(r[5:7]), int(r[8:10])) for r in variables[0]], [1 - r for r in variables[1]], color=COLOR_MODEL, label="Complement of surv_ai Model")
pyplot.plot([*[datetime(2022,i + 1,1) for i in range(12)], *[datetime(2023,i + 1,1) for i in range(5)]], [1.23, 1.05, 0.68, 0.68, 0.82, 0.72, 0.04, -0.4, -0.28, -0.37, -0.6, -1.2, -0.97, -1.24, -0.98, -1.18, -1.21], color=COLOR_CCI, label="Yield Spread (10y - 1y)")

ax1.set_ylabel("Agreement (n_agents=100)")
pyplot.xlabel("Date")
ax2.set_ylabel("Yield Spread (10y - 1y)")

ax1.legend(loc="upper left")
ax2.legend(loc="upper right")
  
# giving a title to my graph
pyplot.title('"The United States economy looks like it is heading for a recession."')
  
# function to show the plot
pyplot.show()

In [ ]:
from surv_ai import (
    GPTClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)

sources = [
    "nytimes.com",
    "cnn.com",
    "wsj.com",
    "foxnews.com",
]

client = GPTClient(os.environ["OPEN_AI_API_KEY"])
model = Model(
    Survey,
    parameters=[
        SurveyParameter(
            independent_variable=source,
            kwargs={
                "client": client,
                "tool_belt": ToolBelt(
                    client,
                    tools=[
                        GoogleCustomSearchTool(
                            client,
                            os.environ["GOOGLE_API_KEY"],
                            os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                            n_pages=20,
                            start_date="2023-05-01",
                            end_date="2023-06-01",
                            only_include_sources=[source]
                        )
                    ]
                ),
                "n_agents": 50,
                "max_knowledge_per_agent": 20,
                "base_knowledge": [
                    Knowledge(
                        text=f"It is currently 2023-06-01. The included articles were published between 2023-05-01 and 2023-06-01",
                        source="Additional context",
                    ),
                ],
            }
        )
        for source in sources
    ],
)
results = await model.build(
    "Republicans are responsible for the impending debt ceiling crisis."
)


In [ ]:
from matplotlib import pyplot

variables = model.get_plot_variables(results)
pyplot.scatter(*variables)
pyplot.ylabel("Agreement (n_agents=100)")
pyplot.title("Republicans are responsible for the impending debt ceiling crisis.")
pyplot.show()

In [ ]:
from surv_ai import (
    GPTClient,
    AnthropicClient,
    Model,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
    SurveyParameter
)


clients = [AnthropicClient(os.environ["ANTHROPIC_API_KEY"]), GPTClient(os.environ["OPEN_AI_API_KEY"])]
model = Model(
    Survey,
    parameters=[
        SurveyParameter(
            independent_variable=client.__class__.__name__,
            kwargs={
                "client": client,
                "tool_belt": ToolBelt(
                    client,
                    tools=[
                        GoogleCustomSearchTool(
                            client,
                            os.environ["GOOGLE_API_KEY"],
                            os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                            n_pages=20,
                            start_date="2023-01-01",
                            end_date="2024-05-01",
                            max_concurrency=3,
                        )
                    ]
                ),
                "n_agents": 100,
                "max_concurrency": 3,
                "max_knowledge_per_agent": 3,
                "base_knowledge": [
                    Knowledge(
                        text=f"It is currently 2023-06-01. The included articles were published between 2023-01-01 and 2023-06-01",
                        source="Additional context",
                    ),
                ],
            }
        )
        for client in clients
    ],
)
results = await model.build(
    "OpenAI has been irresponsible in their handling of AI technology."
)

In [ ]:
from matplotlib import pyplot

variables = model.get_plot_variables(results)
pyplot.scatter(*variables)
pyplot.ylabel("Agreement (n_agents=100)")
pyplot.title("OpenAI has been irresponsible in their handling of AI technology.")
pyplot.show()

In [ ]:
from surv_ai import (
    GPTClient,
    ToolBelt,
    GoogleCustomSearchTool,
    Knowledge,
    Survey,
)
client = GPTClient(os.environ["OPEN_AI_API_KEY"])

survey = Survey(
    client=client,
    tool_belt=ToolBelt(
        client,
        tools=[
            GoogleCustomSearchTool(
                client,
                os.environ["GOOGLE_API_KEY"],
                os.environ["GOOGLE_SEARCH_ENGINE_ID"],
                start_date="2023-01-01",
                end_date="2023-05-23",
                n_pages=10,
            )
        ]
    ),
    max_knowledge_per_agent=3,
    n_agents=10,
    base_knowledge=[
        Knowledge(
            text="It is currently 2023/05/23, all the articles are from 2023.",
            source="Additional context",
        ),
    ],
)
 
await survey.conduct(
    "The price of Bitcoin will increase over the next 30 days",
)